# Metadata analysis of the MAST dataset

Reference: https://mastapp.site/

### Objectives:

* How many shots in the M9 campaign?
* Find label for each shot that tells if the shot is good or not? (how many shot left?)
* For a given shot, find labels for each diagnostic that tells if its a wrong one or not?
* A variable is called efit and refers to the EFIT++ reconstruction. See if there are information about the meaning of the variables.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import xarray as xr

plt.rcParams["font.family"] = "sans"
plt.rcParams["font.size"] = 8
sns.set_palette('muted')

#### 0. Visualization of the whole shots

In [2]:
URL = 'https://mastapp.site'

In [3]:
shots_df = pd.read_parquet(f'{URL}/parquet/level2/shots')
shots_df

,context,url,preshot_description,postshot_description,campaign,current_range,divertor_config,plasma_shape,commissioner,facility,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/11766.zarr,\nRELOAD 10232 PLASMA CONDITIONING SHOT FROM M4\n,"\nGOOD PLASMA, RAN FINE. SL JOINT ALARMS RATHE...",M5,None,Conventional,None,None,MAST,...,8.088068,8.478948,NaN,9286.269388,14370.632,NaN,NaN,NaN,NaN,-0.011828
1,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/11767.zarr,\nREPEAT\n,\nOK BUT LOST VERTICAL CONTROL - FA2 JUST DIED...,M5,None,Conventional,None,None,MAST,...,7.707199,8.559942,NaN,7343.788162,11461.490,NaN,NaN,NaN,NaN,-0.007570
2,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/11768.zarr,\nSELECT ALL 4 FAS AND LOAD EFC SETTINGS FROM ...,\nOK. GOT FA4 BUT NOT FA3\n,M5,None,Conventional,None,None,MAST,...,8.300782,8.580827,NaN,10585.928086,12660.081,NaN,NaN,NaN,NaN,-0.011107
3,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/11769.zarr,"\nREPEAT, WITH TF RAISED TO 0.58.\n",\nSLIDING JOINT ALARMS A BIT LOWER. PLASMA OK.\n,M5,None,Conventional,None,None,MAST,...,8.112973,8.742647,NaN,8189.273726,12494.456,NaN,NaN,NaN,NaN,-0.008156
4,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/11771.zarr,\nRELOADED 11769 AND SET UP PLASMA CURRENT FEE...,\nGOOD PLASMA F/B CONTROL. SLIDING JOINT ALARM...,M5,None,Conventional,None,None,MAST,...,7.978917,8.760519,NaN,8206.258460,12345.816,NaN,NaN,NaN,NaN,-0.008726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11276,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/30467.zarr,\nRepeat with new neutron camera position.\ncH...,\nTwo times lower DD neutron rate than referen...,M9,700 kA,Conventional,Connected Double Null,None,MAST,...,9.029202,9.046394,NaN,49469.122469,52653.445,NaN,NaN,NaN,NaN,0.013202
11277,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/30468.zarr,\nRepeat with new neutron camera position.\ncH...,\nGood beam.\nGood repeat.\n,M9,700 kA,Conventional,Lower Single Null,None,MAST,...,9.102411,9.107017,NaN,48516.962675,49382.133,NaN,NaN,NaN,NaN,0.012445
11278,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/30469.zarr,\nRepeat with increased beam power (74 kV)\ncH...,\nGood shot. Modes present.\n,M9,700 kA,Conventional,Connected Double Null,None,MAST,...,8.988730,9.047923,NaN,47466.249616,49115.805,NaN,NaN,NaN,NaN,0.015299
11279,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/30470.zarr,\nRepeat last using hydrogen in outboard and c...,\nNo HF gas.\n,M9,700 kA,Conventional,None,None,MAST,...,9.687049,10.055509,NaN,17290.432865,22310.516,NaN,NaN,NaN,NaN,0.015164


In [4]:
columns = shots_df.columns
print(f'Number of columns: {len(columns)}\n')
for col in columns:
    print(f"{col.ljust(25)}: {shots_df[col].dtype}")

Number of columns: 290

context                  : object
url                      : object
preshot_description      : object
postshot_description     : object
campaign                 : object
current_range            : object
divertor_config          : object
plasma_shape             : object
commissioner             : object
facility                 : object
type                     : object
title                    : object
shot_id                  : int64
uuid                     : object
endpoint_url             : object
timestamp                : datetime64[ns]
reference_shot           : float64
scenario                 : float64
heating                  : object
pellets                  : object
rmp_coil                 : object
cpf_p03249               : object
cpf_p04673               : object
cpf_p04674               : object
cpf_p04675               : object
cpf_p04676               : object
cpf_p04677               : object
cpf_p04678               : object
cpf_p04679     

In [5]:
print(shots_df.loc[0, 'cpf_zmag_efit'])

-0.01182766


#### 1. M9 campaign selection

In [6]:
shots_df_m9 = pd.read_parquet(f'{URL}/parquet/level2/shots?filters=campaign$eq:M9')
shots_df_m9

,context,url,preshot_description,postshot_description,campaign,current_range,divertor_config,plasma_shape,commissioner,facility,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/28752.zarr,\nChange ZREF to +1cm.\n,\n280ms shot.\n,M9,700 kA,Conventional,Connected Double Null,None,MAST,...,8.654442,8.727527,None,44755.881482,67405.750,None,None,None,None,0.032862
1,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/28647.zarr,\nRepeat after abort.\n,\nGood 400kA 330ms plasma shot.\n,M9,400 kA,Conventional,None,None,MAST,...,6.784744,9.231632,None,7209.400804,14854.986,None,None,None,None,-0.063345
2,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/28750.zarr,\nChange zref to -1 cm\n,\n470ms shot. 400ms SS NBI. Sawteeth from 250m...,M9,700 kA,Conventional,Connected Double Null,None,MAST,...,9.083099,9.108404,None,44471.072260,72576.510,None,None,None,None,0.004665
3,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/28631.zarr,"\nRepeat, with loop volts tweak and +-25kA P1P...",\nFirst plasma.\n,M9,400 kA,Conventional,None,None,MAST,...,5.389849,7.971025,None,1190.965664,3060.270,None,None,None,None,NaN
4,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/28632.zarr,"\nRepeat, with -ve P1PS current limit 45kA. Tw...",\nGood plasma shot.\n,M9,400 kA,Conventional,None,None,MAST,...,8.597655,8.830988,None,5577.911845,24692.621,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/30467.zarr,\nRepeat with new neutron camera position.\ncH...,\nTwo times lower DD neutron rate than referen...,M9,700 kA,Conventional,Connected Double Null,None,MAST,...,9.029202,9.046394,None,49469.122469,52653.445,None,None,None,None,0.013202
1097,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/30468.zarr,\nRepeat with new neutron camera position.\ncH...,\nGood beam.\nGood repeat.\n,M9,700 kA,Conventional,Lower Single Null,None,MAST,...,9.102411,9.107017,None,48516.962675,49382.133,None,None,None,None,0.012445
1098,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/30469.zarr,\nRepeat with increased beam power (74 kV)\ncH...,\nGood shot. Modes present.\n,M9,700 kA,Conventional,Connected Double Null,None,MAST,...,8.988730,9.047923,None,47466.249616,49115.805,None,None,None,None,0.015299
1099,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",s3://mast/level2/shots/30470.zarr,\nRepeat last using hydrogen in outboard and c...,\nNo HF gas.\n,M9,700 kA,Conventional,None,None,MAST,...,9.687049,10.055509,None,17290.432865,22310.516,None,None,None,None,0.015164


In [7]:
print(f"Number of shots in M9 campaign: {len(shots_df_m9)}")

Number of shots in M9 campaign: 1101


Find the list of the M9 campaign shot_id

In [8]:
m9_shot_ids = shots_df_m9['shot_id'].tolist()

print(f"Number of shot IDs in M9 campaign: {len(m9_shot_ids)}")
print("M9 campaign shot_id:", m9_shot_ids[:5])

Number of shot IDs in M9 campaign: 1101
M9 campaign shot_id: [28752, 28647, 28750, 28631, 28632]


#### 2. Find labels for each shot

Interesting columns:
* `cpf_abort`: interupted shot if 1
* `cpf_useful`: good for analysis if 1

In [9]:
# Sélectionner les colonnes spécifiques
abort_values = shots_df_m9['cpf_abort']
useful_values = shots_df_m9['cpf_useful']

# Afficher les 5 premières valeurs (exemple)
print("Valeurs de cpf_abort :\n", abort_values.describe())
print("\nValeurs de cpf_useful :\n", useful_values.describe())

Valeurs de cpf_abort :
 count    2.0
mean     1.0
std      0.0
min      1.0
25%      1.0
50%      1.0
75%      1.0
max      1.0
Name: cpf_abort, dtype: float64

Valeurs de cpf_useful :
 count    755.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: cpf_useful, dtype: float64


In [10]:
good_shots = shots_df_m9.loc[(shots_df_m9['cpf_useful'] == 1) & (shots_df_m9['cpf_abort'] != 1)]
print(f"Number of good shots in M9 campaign: {len(good_shots)}")

Number of good shots in M9 campaign: 755


In [11]:
shots_df_m9['shot_label'] = (shots_df_m9['cpf_useful'] == 1).astype(int)
shots_df_m9[['shot_id','cpf_useful', 'cpf_abort', 'shot_label']]

,shot_id,cpf_useful,cpf_abort,shot_label
0,28752,1.0,NaN,1
1,28647,NaN,NaN,0
2,28750,1.0,NaN,1
3,28631,NaN,NaN,0
4,28632,NaN,NaN,0
...,...,...,...,...
1096,30467,1.0,NaN,1
1097,30468,1.0,NaN,1
1098,30469,1.0,NaN,1
1099,30470,NaN,NaN,0


Find the list of good shot_id and the list of bad shot_id.

In [12]:
good_shot_ids = shots_df_m9.loc[shots_df_m9['shot_label'] == 1, 'shot_id'].tolist()
bad_shot_ids = shots_df_m9.loc[shots_df_m9['shot_label'] == 0, 'shot_id'].tolist()

print(f"Number of good shot IDs: {len(good_shot_ids)}")
print("Good shot IDs:", good_shot_ids[:25])

print("\n")

print(f"Number of bad shot IDs: {len(bad_shot_ids)}")
print("Bad shot IDs:", bad_shot_ids[:5])

Number of good shot IDs: 755
Good shot IDs: [28752, 28750, 28655, 28656, 28657, 28744, 28751, 28747, 28748, 28749, 28755, 28757, 28758, 28763, 28801, 28764, 28765, 28766, 28767, 28768, 28769, 28770, 28771, 28772, 28773]


Number of bad shot IDs: 346
Bad shot IDs: [28647, 28631, 28632, 28652, 28653]


#### 3. Find label for each disgnostics (M9 campaign + magnetics)

Finding a label per diagnostic is quite more difficult.

1. We will first try to see if there are labels in the existant dataset.
2. If not, we will implement unsuvervised technics to label the dataset.

In [13]:
# Shot_id number for example
sh_id = 28752
sh_id = 15585
grp = 'magnetics'

In [14]:
data = xr.open_zarr(
    f"https://s3.echo.stfc.ac.uk/mast/level2/shots/{sh_id}.zarr",
    group=grp,
)
mag_var = data.data_vars
mag_var

Data variables:
    b_field_pol_probe_cc_field        (b_field_pol_probe_cc_channel, time_mirnov) float64 11MB ...
    b_field_pol_probe_ccbv_field      (b_field_pol_probe_ccbv_channel, time) float64 919kB ...
    b_field_pol_probe_obr_field       (b_field_pol_probe_obr_channel, time) float64 414kB ...
    b_field_pol_probe_obv_field       (b_field_pol_probe_obv_channel, time) float64 414kB ...
    b_field_pol_probe_omv_voltage     (b_field_pol_probe_omv_channel, time_mirnov) float64 7MB ...
    b_field_tor_probe_saddle_field    (b_field_tor_probe_saddle_field_channel, time_saddle) float64 3MB ...
    b_field_tor_probe_saddle_voltage  (b_field_tor_probe_saddle_voltage_channel, time_saddle) float64 3MB ...
    flux_loop_flux                    (flux_loop_channel, time) float64 345kB ...
    ip                                (time) float64 23kB ...

In [15]:
for var in mag_var:
    print(f"{var.ljust(35)}: {mag_var[var].attrs}")

b_field_pol_probe_cc_field         : {'description': 'Centre column poloidal mirnov array', 'imas': 'magnetics.b_field_pol_probe[:].field.data', 'name': 'b_field_pol_probe_cc_field', 'units': 'T'}
b_field_pol_probe_ccbv_field       : {'description': 'Centre column Bv array', 'imas': 'magnetics.b_field_pol_probe[:].field.data', 'label': 'CCBV01', 'name': 'b_field_pol_probe_ccbv_field', 'uda_name': 'AMB_CCBV01', 'units': 'T'}
b_field_pol_probe_obr_field        : {'description': 'Outer Br array', 'imas': 'magnetics.b_field_pol_probe[:].field.data', 'label': 'OBR01', 'name': 'b_field_pol_probe_obr_field', 'uda_name': 'AMB_OBR01', 'units': 'T'}
b_field_pol_probe_obv_field        : {'description': 'Outer Bv array', 'imas': 'magnetics.b_field_pol_probe[:].field.data', 'label': 'OBV01', 'name': 'b_field_pol_probe_obv_field', 'uda_name': 'AMB_OBV01', 'units': 'T'}
b_field_pol_probe_omv_voltage      : {'description': 'Outboard vertical mirnov array', 'imas': 'magnetics.b_field_tor_probe[:].volta

In [16]:
signals_df = pd.read_parquet(f"{URL}/parquet/level2/signals?shot_id={sh_id}")
signals_df.head(5)

,context,shape,provenance,description,dimensions,quality,type,title,uuid,name,version,rank,url,endpoint_url,source,units,imas,shot_id
0,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",[115],None,,[time],Not Checked,dcat:Dataset,Signal Dataset,a8042f40-115e-58b8-b12e-439e37a57b8a,beta_normal,0,1,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,equilibrium,,equilibrium,15585
1,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",[115],None,,[time],Not Checked,dcat:Dataset,Signal Dataset,fd40c197-8dc7-53eb-accf-66e8533d1d5c,beta_pol,0,1,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,equilibrium,,equilibrium,15585
2,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",[115],None,,[time],Not Checked,dcat:Dataset,Signal Dataset,7d2a3fe5-029f-5cec-b4a5-0431335dabb1,beta_tor,0,1,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,equilibrium,,equilibrium,15585
3,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",[115],None,,[time],Not Checked,dcat:Dataset,Signal Dataset,c34aa514-20c5-5320-b052-4e5bf4d48768,bphi_rmag,0,1,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,equilibrium,,equilibrium,15585
4,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...",[115],None,,[time],Not Checked,dcat:Dataset,Signal Dataset,80b658d3-3017-59c0-8f27-14027fe80b02,bvac_rmag,0,1,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,equilibrium,,equilibrium,15585


In [17]:
signals_df.loc[4]

context         {'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...
shape                                                       [115]
provenance                                                   None
description                                                      
dimensions                                                 [time]
quality                                               Not Checked
type                                                 dcat:Dataset
title                                              Signal Dataset
uuid                         80b658d3-3017-59c0-8f27-14027fe80b02
name                                                    bvac_rmag
version                                                         0
rank                                                            1
url                             s3://mast/level2/shots/15585.zarr
endpoint_url                           https://s3.echo.stfc.ac.uk
source                                                equilibrium
units     

In [18]:
signals_df.loc[signals_df['imas']=='magnetics'].head(5)

,context,shape,provenance,description,dimensions,quality,type,title,uuid,name,version,rank,url,endpoint_url,source,units,imas,shot_id
39,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...","[5, 287300]",None,,"[b_field_pol_probe_cc_channel, time_mirnov]",Not Checked,dcat:Dataset,Signal Dataset,fae65e38-51b1-5a8b-9962-c15b6c620901,b_field_pol_probe_cc_field,0,2,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,magnetics,A,magnetics,15585
40,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...","[40, 2873]",None,,"[b_field_pol_probe_ccbv_channel, time]",Not Checked,dcat:Dataset,Signal Dataset,e239bbb7-92d4-5a6c-bfcd-8fbd74770a56,b_field_pol_probe_ccbv_field,0,2,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,magnetics,A,magnetics,15585
41,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...","[18, 2873]",None,,"[b_field_pol_probe_obr_channel, time]",Not Checked,dcat:Dataset,Signal Dataset,4807e584-82e1-5ed1-b7f4-00a454e0ee1b,b_field_pol_probe_obr_field,0,2,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,magnetics,A,magnetics,15585
42,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...","[18, 2873]",None,,"[b_field_pol_probe_obv_channel, time]",Not Checked,dcat:Dataset,Signal Dataset,2cca98e2-5864-5884-82a1-dbbed109b893,b_field_pol_probe_obv_field,0,2,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,magnetics,A,magnetics,15585
43,"{'dcat': 'http://www.w3.org/ns/dcat#', 'dct': ...","[3, 287300]",None,,"[b_field_pol_probe_omv_channel, time_mirnov]",Not Checked,dcat:Dataset,Signal Dataset,c7ae1b34-b6da-523f-88ce-ef5e72d5007d,b_field_pol_probe_omv_voltage,0,2,s3://mast/level2/shots/15585.zarr,https://s3.echo.stfc.ac.uk,magnetics,A,magnetics,15585


In [19]:
shot_data = requests.get(f'https://mastapp.site/json/shots/{sh_id}').json()
for key, value in shot_data.items():
    if isinstance(value, list):
        print(f"{key.ljust(30)}: {len(value)} items")
    else:
        print(f"{key.ljust(30)}: {value}")

@context                      : {'dct': 'http://purl.org/dc/terms/', 'dqv': 'http://www.w3.org/ns/dqv#', 'url': 'schema:url', 'dcat': 'http://www.w3.org/ns/dcat#', 'foaf': 'http://xmlns.com/foaf/0.1/', 'uuid': 'dct:identifier', 'title': 'dct:title', 'schema': 'https://schema.org', 'timestamp': 'dct:date', 'sdmx-measure': 'http://purl.org/linked-data/sdmx/2009/measure#'}
@type                         : dcat:Dataset
title                         : Shot Dataset
shot_id                       : 15585
uuid                          : b82d6bf8-451b-59ae-8965-a0915732d890
url                           : s3://mast/level1/shots/15585.zarr
endpoint_url                  : https://s3.echo.stfc.ac.uk
timestamp                     : 2006-06-08T12:57:00
preshot_description           : 
repeat

postshot_description          : 
no beam

campaign                      : M6
reference_shot                : 14900
scenario                      : None
heating                       : None
pellets                

In [20]:
endpoint, url = shot_data["endpoint_url"], shot_data["url"]
shot_url = url.replace("s3:/", endpoint)
print(f"Shot URL: {shot_url}")

Shot URL: https://s3.echo.stfc.ac.uk/mast/level1/shots/15585.zarr


Nothing here seems to be a good label for each diagnostic.

Update: After speaking with Samuel Jackson, there are in fact no label available for each diagnostics. But also, the shots' labels are kind of subjective and hand-crafted.

#### 4. EFIT++ information meaning

In [29]:
sh_id = 28752
sh_id = 15585
grp = 'equilibrium'

In [30]:
data_eq = xr.open_zarr(
    f"https://s3.echo.stfc.ac.uk/mast/level2/shots/{sh_id}.zarr",
    group=grp,
)
eq_var = data_eq.data_vars
eq_var

Data variables:
    beta_normal          (time) float64 920B ...
    beta_pol             (time) float64 920B ...
    beta_tor             (time) float64 920B ...
    bphi_rmag            (time) float64 920B ...
    bvac_rmag            (time) float64 920B ...
    da_rating            (time) float64 920B ...
    elongation           (time) float64 920B ...
    elongation_axis      (time) float64 920B ...
    ip_rating            (time) float64 920B ...
    j_tor                (time, z, major_radius) float64 4MB ...
    lcfs_r               (time, n_boundary_coords) float64 144kB ...
    lcfs_z               (time, n_boundary_coords) float64 144kB ...
    li                   (time) float64 920B ...
    magnetic_axis_r      (time) float64 920B ...
    magnetic_axis_z      (time) float64 920B ...
    minor_radius         (time) float64 920B ...
    psi                  (time, z, major_radius) float64 4MB ...
    q                    (time, profile_r) float64 60kB ...
    q100           

In [31]:
eq_coord = data_eq.coords
eq_coord

Coordinates:
  * major_radius       (major_radius) float64 520B 0.06 0.09 0.12 ... 1.95 1.98
  * n_boundary_coords  (n_boundary_coords) float32 628B 0.0 1.0 ... 155.0 156.0
  * n_x_points         (n_x_points) <U16 256B 'EFM_XPOINT1_R(C)' ... 'EFM_XPO...
  * profile_r          (profile_r) float32 260B 0.0 0.01562 ... 0.9844 1.0
  * time               (time) float64 920B -0.0656 -0.0606 ... 0.4994 0.5044
  * z                  (z) float32 260B -2.0 -1.938 -1.875 ... 1.875 1.938 2.0

In [32]:
data_eq["psi"].attrs

{'description': '',
 'imas': 'equilibrium.time_slice[:].profiles_2d[:].psi',
 'label': 'psi(r,z)',
 'name': 'psi',
 'uda_name': 'EFM_PSI(R,Z)',
 'units': 'Wb / rad'}

In [34]:
data_eq["psi"].values

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan